In [1]:
import os
os.chdir("../scripts/")

In [2]:
import cv2
import numpy as np
import rosbag
import yaml
import data_conversion
import json

In [3]:
DATA_PATH = "/scratchdata/processed/indoor_lift"

# Open bag file
bag_file_path = "/scratchdata/indoor_lift.bag"
bag = rosbag.Bag(bag_file_path)

config = yaml.load(open("../config/gemini2L.yaml", "r"), Loader=yaml.FullLoader)

In [4]:
if os.path.exists(DATA_PATH):
    os.system('rm -r {}'.format(DATA_PATH))
    
os.system('mkdir {}'.format(DATA_PATH))
os.system('mkdir {}'.format(DATA_PATH+'/rgb'))
os.system('mkdir {}'.format(DATA_PATH+'/depth'))


0

In [5]:
for topic, msg, t in bag.read_messages(topics=["/camera/color/camera_info"]):
    D = msg.D
    K = msg.K
    R = msg.R
    P = msg.P
    break

fx = P[0]
fy = P[5]
cx = P[2]
cy = P[6]

print(fx, fy, cx, cy)

306.9346923828125 306.8908386230469 318.58868408203125 198.37969970703125


In [6]:
data = {}
data["D"] = D
data["K"] = K
data["R"] = R
data["P"] = P

with open(DATA_PATH+"/camera_info.json", "w") as f:
    json.dump(data, f, indent=4)

In [7]:
rgb = []

for topic, msg, t in bag.read_messages(topics=["/camera/color/image_raw"]):
    rgb.append(msg)

depth = []

for topic, msg, t in bag.read_messages(topics=["/camera/depth/image_raw"]):
    depth.append(msg)


In [8]:
pt = 0
diff = 1e-5 # 0.1 ms

cnt = 0

for i in range(len(rgb)):
    rgb_msg, rgb_t = rgb[i], rgb[i].header.stamp
    depth_msg, depth_t = depth[pt], depth[pt].header.stamp
    
    while depth_t.to_nsec() - rgb_t.to_nsec() < diff * 1e9 and pt < len(depth):
        depth_msg, depth_t = depth[pt], depth[pt].header.stamp

        if abs(rgb_t.to_nsec() - depth_t.to_nsec()) < diff:
            pt += 1
            break
            
        pt += 1

    rgb_path = DATA_PATH + '/rgb/' + str(cnt) + '.png'
    depth_path = DATA_PATH + '/depth/' + str(cnt) + '.png'

    rgb_img = data_conversion.topic_to_image(rgb_msg)
    rgb_img = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2RGB)
    depth_img = data_conversion.topic_to_depth(depth_msg,config["depth_anything_config"])

    depth_img = np.array(depth_img * 1000, dtype=np.uint16)

    cv2.imwrite(rgb_path, rgb_img)
    cv2.imwrite(depth_path, depth_img)
    
    #with open(rgb_path, 'wb') as f:
    #    f.write(rgb_msg.data)
    
    #with open(depth_path, 'wb') as f:
    #    f.write(depth_msg.data)
    
    cnt += 1    